
## Summary Review:

This project examined Student placement data and determined the best placement for a student based on their GPA and test scores.
The project utilized various data analysis techniques, including data visualization and statistical analysis, to identify the key factors that influence student placement.
The results of the analysis were then used to develop a predictive model that can be used to determine the best placement for a student based on their GPA
and test scores. The project demonstrates the application of data analysis and machine learning techniques to real-world problems.


In [ ]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import stats 
from sklearn.model_selection import train_test_split,
from sklearn.linear_model import LogisticRegression,
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [ ]:
# Function for loading the dataset in CSV format
def load_data(file_path):
    """
    Loads a dataset from a CSV file.

    Parameters
    ----------
    file_path : str
        The path to the CSV file containing the dataset

    Returns
    -------
    pandas.DataFrame
        The loaded dataset
    """
    # Load the dataset from the specified file path
    data = pd.read_csv(file_path)
    return data


df = load_data("Placement.csv")



In [ ]:
# Initial and expression analyses about the dataset
def initial_analysis(data):
    """
    Function to perform initial and exploratory analysis about the dataset.

    Parameters
    ----------
    data : pandas DataFrame
        The dataset to be analyzed.

    Returns
    -------
    None

    Notes
    -----
    This function does the following:
        1. Displays the first few rows of the dataset.
        2. Displays the summary statistics of the dataset.
        3. Displays the information about the dataset.
        4. Displays the column names of the dataset.
        5. Displays the unique values and their counts for each column.
        6. Displays the min minium and maximum values for each column.
    """
    # Display the first few rows of the dataset
    print("First few rows of the dataset:")
    print(data.head())

    # Display the summary statistics of the dataset
    print("\n==== Summary statistics of the dataset: ====")
    print(data.describe())

    # Display the information about the dataset
    print("\n==== Information about the dataset: ====")
    print(data.info())

    # Display the column names of the dataset
    print("\n==== Column names of the dataset: ====")
    print(data.columns)

    # Display the unique values and their counts for each column
    print("\n==== Unique values and their counts for each column: ====")
    print(data.nunique())

    # Display the min minium and maximum values for each column
    print("\n==== Minium values for each column: ====")
    print(data.min())
    print("\n==== Maximum values for each column: ====")
    print(data.max())


initial_analysis(df)



In [ ]:
# Manage null values in the dataset
def manage_null_values(data):
    """
    Check for null values in the dataset, fill them in with the mean for numeric
    columns and the mode for categorical columns, and check for null values after
    filling.

    Parameters
    ----------
    data : pandas.DataFrame
        The dataset to manage null values for

    Returns
    -------
    None
    """
    # Check for null values in the dataset
    print("Null values in the dataset:")
    print(data.isnull().sum())

    # Fill null values in the numeric columns with the mean of the respective columns
    numeric_columns = data.select_dtypes(include=["int64", "float64"]).columns
    data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

    # Fill null values in the categorical columns with the mode of the respective columns
    categorical_columns = data.select_dtypes(include=["object", "category"]).columns
    data[categorical_columns] = data[categorical_columns].fillna(
        data[categorical_columns].mode().iloc[0]
    )

    # Check for null values after filling
    print("\nNull values after filling:")
    print(data.isnull().sum())


manage_null_values(df)



In [ ]:
# Definition for plotting the numeric data(bar plot, histogram, box plot, scatter plot, pair plot)
def plot_data(data):
    """
    Function to plot the data using different types of plots.

    Parameters
    ----------
    data : pandas DataFrame
        The dataset to be plotted.

    Returns
    -------
    None

    Notes
    -----
    This function plots the data using different types of plots, including bar plot, histogram, box plot, scatter plot, and pair plot.
    All plots are displayed on a single page.
    """
    numeric_columns = ["CGPA", "Internships", "Salary (INR LPA)"]
    n_cols = len(numeric_columns)

    # Create subplots
    fig, axes = plt.subplots(n_cols, 2, figsize=(12, 4 * n_cols))
    plt.subplots_adjust(hspace=0.4, wspace=0.4)

    # Plot histograms and box plots
    for i, column in enumerate(numeric_columns):
        # Histogram
        sns.histplot(data[column], kde=True, bins=10, ax=axes[i, 0])
        axes[i, 0].set_title(f"Histogram of {column}")
        axes[i, 0].set_xlabel(column)
        axes[i, 0].set_ylabel("Frequency")

        # Box plot
        data[column].plot(kind="box", ax=axes[i, 1])
        axes[i, 1].set_title(f"Box plot of {column}")
        axes[i, 1].set_xlabel(column)
        axes[i, 1].set_ylabel("Values")

    plt.tight_layout()
    plt.show()

    # Create pair plot in a separate figure
    plt.figure()
    sns.pairplot(data.select_dtypes(include=["int64", "float64"]))
    plt.tight_layout()
    plt.show()


# Data visualization
plot_data(df)



In [ ]:
# Outlier detection and removal
def detect_outliers(data):
    """
    Detect outliers in the dataset using the Interquartile Range (IQR) method.

    Parameters
    ----------
    data : pandas.DataFrame
        The dataset to detect outliers in

    Returns
    -------
    pandas.DataFrame
        The cleaned dataset with outliers removed
    """
    # Create a copy of the data to avoid modifying the original
    data_clean = data.copy()

    # Process each numeric column
    for column in data.select_dtypes(include=["int64", "float64"]).columns:
        # Calculate quartiles and IQR
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1

        # Calculate outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Find outliers
        outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]

        # Print outlier information
        print(f"\nColumn: {column}")
        print(f"Lower bound: {lower_bound:.2f}")
        print(f"Upper bound: {upper_bound:.2f}")
        print(f"Number of outliers: {len(outliers)}")

        if len(outliers) > 0 and len(outliers) < 10:
            print(f"Outlier values: {sorted(outliers[column].tolist())}")

        # Remove outliers from the cleaned dataset
        data_clean = data_clean[
            (data_clean[column] >= lower_bound) & (data_clean[column] <= upper_bound)
        ]

    return data_clean


# Detect and remove outliers
df_clean = detect_outliers(df)
print("\nCleaned dataset:")
print(df_clean)



In [ ]:
# Feature engineering
df_clean["CGPA_grade"] = pd.cut(
    df_clean["CGPA"],
    bins=[0, 3.0, 5.0, 8.0, 10.0],
    labels=["poor", "average", "good", "excellent"],
)

df_clean["Salary (INR LPA)_grade"] = pd.cut(
    df_clean["Salary (INR LPA)"],
    bins=[0, 5, 10, 15, 20, 25, 30],
    labels=["Entry Level", "Junior", "Mid-Level", "Senior", "Expert", "Executive"],
)



In [ ]:
# Dfinition for plotting the categorical data
def plot_categorical_data(data):
    """
    Function to plot the categorical data using different types of plots.

    Parameters
    ----------
    data : pandas DataFrame
        The dataset to be plotted.

    Returns
    -------
    None

    Notes
    -----
    This function plots the categorical data using different types of plots, including bar plot, count plot, and box plot.
    All plots are displayed on a single page.
    """

    categorical_columns = [
        col
        for col in data.columns
        if data[col].dtype == "object" or data[col].dtype == "category"
    ]
    n_cols = len(categorical_columns)

    range_info = {
        "CGPA_grade": {
            "Poor": "0-3.0",
            "Average": "3.0-5.0",
            "Good": "5.0-8.0",
            "Excellent": "8.0-10.0",
        },
        "Salary (INR LPA)_grade": {
            "Entry Level": "0-5 LPA",
            "Junior": "5-10 LPA",
            "Mid-Level": "10-15 LPA",
            "Senior": "15-20 LPA",
            "Expert": "20-25 LPA",
            "Executive": "25-30 LPA",
        },
    }

    # Create subplots
    fig, axes = plt.subplots(n_cols, 1, figsize=(8, 4 * n_cols))
    plt.subplots_adjust(hspace=0.4)

    # count plots
    for i, column in enumerate(categorical_columns):
        # Count plot
        sns.countplot(
            x=column, hue=column, data=data, ax=axes[i], palette="viridis", legend=False
        )
        axes[i].set_title(f"Count plot of {column}")
        axes[i].set_xlabel(column)
        axes[i].set_ylabel("Count")
        if column in range_info.keys():
            legend = axes[i].legend(
                range_info[column].values(),
                loc="upper left",
                title=column,
                bbox_to_anchor=(1.05, 1),
                fontsize=10,
            )
            legend.get_title().set_fontsize(12)

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    plt.close()


plot_categorical_data(df_clean)



In [ ]:
# Shapiro test for normality in numeric columns
def normality_test(data):
    """
    Perform Shapiro test for normality in numeric columns.

    Parameters
    ----------
    data : pandas.DataFrame
        The dataset to perform Shapiro test on

    Returns
    -------
    None
    """
    print(
        "=" * 60
        + "\n Shapiro test for normality in numeric columns \n"
        + "=" * 60
        + "\n"
    )
    numeric_columns = ["CGPA", "Internships", "Salary (INR LPA)"]
    for column in numeric_columns:
        print(f"Shapiro test for {column}:")
        statistic, p_value = stats.shapiro(data[column])
        print(f"Statistic: {statistic:.4f}")
        print(f"P-value: {p_value:.2e}")
        print(
            f"Normality: {'Normal(p-value > 0.05)' if p_value > 0.05 else 'Not normal(p-value <= 0.05)'}"
        )
        print("\n")


normality_test(df_clean)



The Shapiro-Wilk test checks if your data follows a normal distribution. For all three variables (CGPA, Internships, and Salary), the test indicates non-normality because all p-values are below 0.05.

 - **CGPA** (Statistic: 0.9968, P-value: 0.0394) is borderline non-normal. The test statistic is close to 1, suggesting it's nearly normal but doesn't quite meet the statistical threshold.

 - **Internships** (Statistic: 0.8839, P-value: 1.32e-26) shows strong evidence of non-normality with a very low p-value.

 - **Salary** (Statistic: 0.8858, P-value: 2.06e-26) also demonstrates strong non-normality, likely due to a skewed distribution.

Since none of these variables are normally distributed, you should use non-parametric statistical methods instead of those that assume normality (like t-tests or ANOVA).Consider using tests like Mann-Whitney U or Kruskal-Wallis for your analyses.



In [ ]:
# Spearman correlation between CGPA, internships and salary
def spearman_correlation(data):
    """
    Calculate Spearman correlation between CGPA, internships and salary.

    Parameters
    ----------
    data : pandas.DataFrame
        The dataset to calculate Spearman correlation on

    Returns
    -------
    None
    """
    print(
        "=" * 60
        + "\n Spearman correlation between CGPA, internships and salary \n"
        + "=" * 60
        + "\n"
    )
    correlation = data[["CGPA", "Internships", "Salary (INR LPA)"]].corr(
        method="spearman"
    )
    print("Spearman correlation between CGPA, internships and salary")
    ax = sns.heatmap(correlation, annot=True, cmap="coolwarm")
    ax.set_title("Spearman correlation heatmap")
    plt.show()

    print(correlation)


spearman_correlation(df_clean)

The Spearman correlation heatmap shows the correlation between CGPA, Internships, and Salary (INR LPA).
The Spearman correlation results indicate that:

- There is a weak negative correlation between CGPA and Internships. (correlation coefficient: -0.020540)
- There is a weak negative correlation between CGPA and Salary (INR LPA). (correlation coefficient: -0.065308)
- There is a near-zero correlation between Internships and Salary (INR LPA). (correlation coefficient: -0.001089)

Overall, the results suggest that there is no significant correlation between the variables CGPA, Internships, and Salary (INR LPA).



In [ ]:
# Mann-Whitney U test between 'CGPA', 'Internships' and 'Salary' with 'Placed'
df_clean["placed_binary"] = df_clean["Placed"].map({"Yes": 1, "No": 0})


def mann_whitney_test(data):
    """
    Perform Mann-Whitney U test between 'CGPA', 'Internships' and 'Salary' with 'Placed'.

    Parameters
    ----------
    data : pandas.DataFrame
        The dataset to perform Mann-Whitney U test on

    Returns
    -------
    None
    """
    print(
        "=" * 60
        + "\n Mann-Whitney U test between 'CGPA', 'Internships' and 'Salary' with 'Placed' \n"
        + "=" * 60
        + "\n"
    )
    for column in ["CGPA", "Internships", "Salary (INR LPA)"]:
        print(f"Mann-Whitney U test between {column} and Placed:")
        statistic, p_value = stats.mannwhitneyu(data[column], data["placed_binary"])
        print(f"Statistic: {statistic:.4f}")
        print(f"P-value: {p_value:.2e}")
        print(
            f"Significance: {'Significant(p-value < 0.05)\n' if p_value < 0.05 else 'Not significant(p-value >= 0.05)\n'}"
        )


mann_whitney_test(df_clean)


The results of the Mann-Whitney U test indicate a significant relationship between the variables 'CGPA', 'Internships', and 'Salary (INR LPA)' and the variable 'Placed'."
The extremely low p-values (0.00e+00, 2.56e-105, and 3.18e-80) suggest that these relationships are statistically significant."
These findings suggest that there is a strong association between these variables and the likelihood of being placed in a job."
Therefore, it is recommended to further investigate these relationships and explore the potential predictive power of these variables in determining job placement outcomes."


In [ ]:
# Generating Logistic Regression Model

X = df_clean[["CGPA", "Internships", "Salary (INR LPA)"]]
y = df_clean["Placed"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification)


**Summary Report of Logistic Regression Model Results**

Overall Model Performance:
- Overall Accuracy: 1.00 (100%)
- Average of Precision, Recall, and F1-Score: 1.00 (100%)

Confusion Matrix Analysis:
- Out of 93 samples in the "No" class, all 93 were correctly predicted (False Positives = 0).
- Out of 205 samples in the "Yes" class, all 205 were correctly predicted (False Negatives = 0).

Detailed Performance for Each Class:
1. "No" Class:
   - Precision: 1.00 (100%)
   - Recall: 1.00 (100%)
   - F1-Score: 1.00 (100%)

2. "Yes" Class:
   - Precision: 1.00 (100%)
   - Recall: 1.00 (100%)
   - F1-Score: 1.00 (100%)

Conclusion:
The logistic regression model has demonstrated perfect performance on the test data.
All evaluation metrics (precision, recall, and F1-score) for both classes have reached 1.00 (100%), indicating the model's complete ability to distinguish between the two classes.
Given the absence of errors in the confusion matrix, it can be concluded that the model is not overfitting on this dataset and maintains a good balance between the two classes.



In [ ]:
# Random Forest Classifier Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification)

**Summary Report of Random Forest Classifier Model Results**

Overall Model Performance:
- Overall Accuracy: 1.00 (100%)
- Average of Precision, Recall, and F1-Score: 1.00 (100%)

Confusion Matrix Analysis:
- Out of 93 samples in the "No" class, all 93 were correctly predicted (False Positives = 0).
- Out of 205 samples in the "Yes" class, all 205 were correctly predicted (False Negatives = 0).

Detailed Performance for Each Class:
1. "No" Class:
   - Precision: 1.00 (100%)
   - Recall: 1.00 (100%)
   - F1-Score: 1.00 (100%)

2. "Yes" Class:
   - Precision: 1.00 (100%)
   - Recall: 1.00 (100%)
   - F1-Score: 1.00 (100%)

Conclusion:
The random forest classifier model has demonstrated perfect performance on the test data.
All evaluation metrics (precision, recall, and F1-score) for both classes have reached 1.00 (100%), indicating the model's complete ability to distinguish between the two classes.
Given the absence of errors in the confusion matrix, it can be concluded that the model is not overfitting on this dataset and maintains a good balance between the two classes.




In [ ]:
# Cross Validation for Logistic Regression and Random Forest

# Define logistic regression model
logreg = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Define 5-fold stratified cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg_scores = cross_val_score(logreg, X, y, cv=skf, scoring="accuracy")

# Print out the cross-validation accuracy scores for logistic regression model
print("Logistic Regression CV Accuracy Scores:", logreg_scores)

# Calculate and print out the mean accuracy for logistic regression model
print("Mean Accuracy:", np.mean(logreg_scores))
print("Standard Deviation:", np.std(logreg_scores))
rf_scores = cross_val_score(rf, X, y, cv=skf, scoring="accuracy")

# Print out the cross-validation accuracy scores for random forest model
print("\nRandom Forest CV Accuracy Scores:", rf_scores)
print("Mean Accuracy:", np.mean(rf_scores))
print("Standard Deviation:", np.std(rf_scores))


**Short Report: Cross-Validation Results**

Summary:

A 5-fold stratified cross-validation was performed to compare a Logistic Regression model and a Random Forest classifier. Both models were evaluated based on accuracy.

Results:
- Logistic Regression: Achieved a perfect mean accuracy of 1.0 with a standard deviation of 0.0.
- Random Forest: Also achieved a perfect mean accuracy of 1.0 with a standard deviation of 0.0.

Conclusion:
- Both models demonstrated flawless and identical performance on this dataset, correctly classifying all samples in every validation fold with no variance in their results.

___________________________________________________________________________________________________________________

# Student Placement Analysis - Project Summary

**Project Overview**

This project analyzed student placement data to predict job placement outcomes using CGPA, internship experience, and salary expectations through machine learning techniques.

**Data Processing & Analysis**
- Data Cleaning: Handled missing values using mean/mode imputation and removed outliers using IQR method
- Feature Engineering: Created categorical grades for CGPA and salary levels
- Statistical Testing: Applied Shapiro-Wilk tests (confirmed non-normal distributions) and Mann-Whitney U tests (showed significant relationships between features and placement)
- Correlation Analysis: Used Spearman correlation revealing weak correlations between variables

**Machine Learning Models**
- Models Used: Logistic Regression and Random Forest Classifier
- Data Split: 70/30 train-test split with stratified sampling
- Validation: 5-fold stratified cross-validation

**Key Results**
- Perfect Performance: Both models achieved 100% accuracy on test data
- Cross-Validation: Mean accuracy of 1.0 with zero standard deviation across all folds
- Statistical Significance: All features showed significant relationships with placement outcomes (p < 0.001)

**Technical Implementation**
- Libraries: pandas, numpy, matplotlib, seaborn, scipy, scikit-learn
- Code Quality: Modular functions with comprehensive documentation and reproducible results

**Conclusions**

The project successfully developed highly accurate predictive models for student placement outcomes. The perfect performance suggests strong predictive power of academic performance, internship experience, and salary expectations for job placement success. Results can effectively support career counseling and placement decision-making processes.

